In [1]:
import pandas as pd
import numpy as np
import requests
import bs4

In [2]:
# new list to append urls
post_urls = []
url = "https://www.whitehouse.gov"

# 50 pages of posts
for i in np.arange(50):
    #grab url and append page number
    page_url = url + "/blog?page=" + str(i)
    r = requests.get(page_url)
    soup = bs4.BeautifulSoup(r.content.decode('utf-8'), "lxml")
    # h3 field-content is the tag to get post urls
    page_posts = soup.find_all("h3", "field-content")
    # dates for the posts
    post_dates = soup.find_all("span", "field-content")
    # append page's post urls to our list
    post_urls.append([(post.text, post.find('a')['href'], date.text) for post, date in zip(page_posts, post_dates)])

# flatten list
all_posts = [p for post in post_urls for p in post]

In [4]:
posts = []

# loop through every post and get text in post
for post_data in all_posts:
    req = requests.get(url + post_data[1], allow_redirects=False)
    if req.status_code == 200:
        req_soup = bs4.BeautifulSoup(req.text, "lxml")
        post_body = req_soup.find("div","pane-entity-field").text\
        .encode('ascii', 'ignore').decode('UTF-8').replace("\n", "").replace("\t", "")
        posts.append(post_body)

In [9]:
df_out = pd.DataFrame({'a': [post[2].strip() for post in all_posts], 'b': [post[0] for post in all_posts], 'c': [post.strip() for post in posts]})
df_out.columns = ['Date', 'Title', 'Body']

In [10]:
df_out.to_csv('WH_posts.csv', index=False)

In [11]:
# Everything above was used to create our CSV file
# There is no need to run the cells above
# You can start from the cell below

In [12]:
df = pd.read_csv('WH_posts.csv')